In [ ]:
import numpy as np
import pandas as pd
import math
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import plotly.express as px
import plotly.graph_objs as go

**Data Source:**
https://www.fao.org/aquastat/statistics/query/index.html 

# Initial Data


*   x1: Total Population *(in 1000s)*
*   x2: Population Density *(inhabitants/square km)*
*   x3: GDP per capita *(USD/inhabitant)*
*   x4: Long term average annual precipitation in depth *(mm/year)*
*   x5: Total renewable resources per capita *($m^{3}$/inhabitant /year)*
*   x6: Total water withdrawal per capita *($m^{3}$/inhabitant /year)*
*   x7: Agricultural water withdrawal *(%age of x6)*
*   x8: Industrial water withdrawal *(%age of x6)*
*   x9: Municipal water withdrawal *(%age of x6)*
*   x10: Collected Municipal wastewater  *($km^{3}$/year)*
*   x11: Capacity of the municipal wastewater treatment facilities *($km^{3}$/year)*
*   x12: Not treated municipal wastewater *($km^{3}$/year)*
*   y: Water Stress *(%age)*

In [ ]:
#Read and prepare the data
data = pd.read_csv('https://www.dropbox.com/s/j07u99pg44kbfvm/final_data.csv?dl=1')
data

,Area,Variable Name,Year,Value
0,Afghanistan,Total population,1962,9351.441000
1,Afghanistan,Total population,1967,10399.926000
2,Afghanistan,Total population,1972,11791.215000
3,Afghanistan,Total population,1977,13171.306000
4,Afghanistan,Total population,1982,12882.528000
...,...,...,...,...
18698,Zimbabwe,SDG 6.4.2. Water Stress,2002,39.476155
18699,Zimbabwe,SDG 6.4.2. Water Stress,2007,33.514833
18700,Zimbabwe,SDG 6.4.2. Water Stress,2012,32.488030
18701,Zimbabwe,SDG 6.4.2. Water Stress,2017,31.346226


In [ ]:
#data_x1_x5.pivot(columns='Variable Name', index='Area')['Value']
data = data.groupby(['Year','Variable Name', 'Area'])['Value'].aggregate('mean').unstack(1)
data

Variable Name                            Agricultural water withdrawal as % of total water withdrawal  \
Year Area                                                                                               
1962 Afghanistan                                                                       NaN              
     Albania                                                                           NaN              
     Algeria                                                                           NaN              
     Andorra                                                                           NaN              
     Angola                                                                            NaN              
...                                                                                    ...              
2018 Venezuela (Bolivarian Republic of)                                          73.852110              
     Viet Nam                                                                    94.782397              
     Yemen                                                                       90.743338              
     Zambia                                                                      73.282443              
     Zimbabwe                                                                    80.635470              

Variable Name                            Capacity of the municipal wastewater treatment facilities  \
Year Area                                                                                            
1962 Afghanistan                                                                       NaN           
     Albania                                                                           NaN           
     Algeria                                                                           NaN           
     Andorra                                                                           NaN           
     Angola                                                                            NaN           
...                                                                                    ...           
2018 Venezuela (Bolivarian Republic of)                                                NaN           
     Viet Nam                                                                        0.423           
     Yemen                                                                             NaN           
     Zambia                                                                            NaN           
     Zimbabwe                                                                          NaN           

Variable Name                            Collected municipal wastewater  \
Year Area                                                                 
1962 Afghanistan                                                    NaN   
     Albania                                                        NaN   
     Algeria                                                        NaN   
     Andorra                                                        NaN   
     Angola                                                         NaN   
...                                                                 ...   
2018 Venezuela (Bolivarian Republic of)                             NaN   
     Viet Nam                                                   0.19700   
     Yemen                                                          NaN   
     Zambia                                                         NaN   
     Zimbabwe                                                   0.00027   

Variable Name                            GDP per capita  \
Year Area                                                 
1962 Afghanistan                                    NaN   
     Albania                                        NaN   
     Algeria                                        NaN   
     Andorra                        

In [ ]:
data.isna().sum()

Variable Name
Agricultural water withdrawal as % of total water withdrawal    1180
Capacity of the municipal wastewater treatment facilities       2206
Collected municipal wastewater                                  2170
GDP per capita                                                   409
Industrial water withdrawal as % of total water withdrawal      1204
Long-term average annual precipitation in volume                 210
Municipal water withdrawal as % of total withdrawal             1182
Not treated municipal wastewater                                2082
Population density                                                27
SDG 6.4.2. Water Stress                                         1104
Total population                                                  11
Total renewable water resources per capita                        41
Total water withdrawal per capita                               1090
dtype: int64

## Exploratory Data Analysis



*   Three variables, Capacity of the municipal wastewater treatment facilities (x11), Collected municipal wastewater (x10) and Not treated municipal wastewater (x12) are missing around 75% to 80% of the values. Therefore it can be assumed that they do not impact the water stress (y) values and can be droped from the dataset.
*   All the rows with NaN values can be dropped.
*   Variables x7, x8 and x9 sum up to 100 as they are percentages of x6. Therefore we need only two of these three variables and one of them should be dropped to avoid redundancy.
*   Total population and Population density are related variables. So one of them can be dropped. Total population being a more absolute variable can be dropped.





# Cleaning the dataset

In [ ]:
#drop the rows that are missing "y" values i.e. missing SDG 6.4.2. Water Stress
print("Dimensions of the original dataset: ", data.shape)
data.dropna(subset=["Population density","GDP per capita","SDG 6.4.2. Water Stress","Agricultural water withdrawal as % of total water withdrawal","Municipal water withdrawal as % of total withdrawal"], inplace=True)
data.drop(["Total population","Capacity of the municipal wastewater treatment facilities","Collected municipal wastewater","Not treated municipal wastewater","Industrial water withdrawal as % of total water withdrawal"], axis=1, inplace=True)
print("Dimensions of the new dataset: ", data.shape)

Dimensions of the original dataset:  (2431, 13)
Dimensions of the new dataset:  (1205, 8)


In [ ]:
data.isna().sum()

Variable Name
Agricultural water withdrawal as % of total water withdrawal    0
GDP per capita                                                  0
Long-term average annual precipitation in volume                0
Municipal water withdrawal as % of total withdrawal             0
Population density                                              0
SDG 6.4.2. Water Stress                                         0
Total renewable water resources per capita                      0
Total water withdrawal per capita                               0
dtype: int64

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data

Variable Name,Year,Area,Agricultural water withdrawal as % of total water withdrawal,GDP per capita,Long-term average annual precipitation in volume,Municipal water withdrawal as % of total withdrawal,Population density,SDG 6.4.2. Water Stress,Total renewable water resources per capita,Total water withdrawal per capita
0,1972,Denmark,31.947484,4655.290175,30.29227,60.831510,115.816802,24.716063,1202.273258,183.146293
1,1972,Ghana,54.727904,493.336524,283.14698,32.902354,38.692173,1.505958,6089.084168,37.430171
2,1972,Hungary,22.379496,782.184870,54.79467,20.569268,112.072299,5.828256,9974.980064,323.626719
3,1972,Israel,78.318250,2762.062183,9.60045,16.779789,135.771726,135.122077,594.029932,531.022713
4,1972,Italy,59.201104,2674.044383,250.71488,16.811125,180.058210,37.599562,3525.699205,786.449189
...,...,...,...,...,...,...,...,...,...,...
1200,2018,Venezuela (Bolivarian Republic of),73.852110,6433.928949,1864.23020,22.641793,31.672735,7.540367,45868.196340,783.266091
1201,2018,Viet Nam,94.782397,2538.068147,603.16983,1.470194,288.458056,18.130315,9253.347619,858.539684
1202,2018,Yemen,90.743338,922.404203,88.17099,7.433380,53.977845,169.761905,73.687616,125.093500
1203,2018,Zambia,73.282443,1549.354235,767.66220,18.447837,23.055378,2.835498,6039.751245,90.596269


In [ ]:
data.corr()

Variable Name,Year,Agricultural water withdrawal as % of total water withdrawal,GDP per capita,Long-term average annual precipitation in volume,Municipal water withdrawal as % of total withdrawal,Population density,SDG 6.4.2. Water Stress,Total renewable water resources per capita,Total water withdrawal per capita
Variable Name,,,,,,,,,
Year,1.000000,-0.060807,0.223303,0.024027,0.078427,0.023163,0.030757,-0.068900,-0.051328
Agricultural water withdrawal as % of total water withdrawal,-0.060807,1.000000,-0.415532,0.018947,-0.675577,-0.180608,0.085287,-0.089993,0.296968
GDP per capita,0.223303,-0.415532,1.000000,0.026815,0.250539,0.165558,0.170544,0.093298,0.035759
Long-term average annual precipitation in volume,0.024027,0.018947,0.026815,1.000000,-0.094149,-0.073010,-0.066559,0.054745,0.066076
Municipal water withdrawal as % of total withdrawal,0.078427,-0.675577,0.250539,-0.094149,1.000000,0.148729,-0.011202,0.099943,-0.422910
Population density,0.023163,-0.180608,0.165558,-0.073010,0.148729,1.000000,0.045586,-0.090589,-0.069448
SDG 6.4.2. Water Stress,0.030757,0.085287,0.170544,-0.066559,-0.011202,0.045586,1.000000,-0.078604,0.079976
Total renewable water resources per capita,-0.068900,-0.089993,0.093298,0.054745,0.099943,-0.090589,-0.078604,1.000000,0.094333
Total water withdrawal per capita,-0.051328,0.296968,0.035759,0.066076,-0.422910,-0.069448,0.079976,0.094333,1.000000


There is no direct linear relation between any of the variables and the water stress values. We need to build a prediction model that analyze complex relations between variables such as Neural Networks.

# Scaling the dataset

In [ ]:
#Create X_data and Y_data
X_data= data[["Agricultural water withdrawal as % of total water withdrawal", "GDP per capita", "Long-term average annual precipitation in volume", "Municipal water withdrawal as % of total withdrawal", "Total renewable water resources per capita", "Total water withdrawal per capita", "Population density"]]
Y_data= data["SDG 6.4.2. Water Stress"]
X_scale= X_data.apply(lambda x: (x-x.min(axis=0)) / (x.max(axis=0)-x.min(axis=0)))
#Change the percentage to proportion
Y_scale=Y_data.apply(lambda x: x/100)

# Neural Network Model

## Splitting the dataset

In [ ]:
print(X_scale.shape,Y_scale.shape)

(1205, 7) (1205,)


In [ ]:
#convert dataframes to array
X=pd.DataFrame(X_scale).to_numpy()
Y=pd.DataFrame(Y_scale).to_numpy()

In [ ]:
X

array([[3.20902724e-01, 3.84416071e-02, 2.01606512e-03, ...,
        1.64310175e-03, 3.06699357e-02, 1.43787686e-02],
       [5.49725098e-01, 3.78738908e-03, 1.88773038e-02, ...,
        8.34868443e-03, 5.25083958e-03, 4.67859864e-03],
       [2.24795207e-01, 6.19246477e-03, 3.64997106e-03, ...,
        1.36808319e-02, 5.51757034e-02, 1.39078125e-02],
       ...,
       [9.11489140e-01, 7.35999131e-03, 5.87562111e-03, ...,
        9.44794555e-05, 2.05430567e-02, 6.60111833e-03],
       [7.36099775e-01, 1.25802472e-02, 5.11864775e-02, ...,
        8.28099080e-03, 1.45252708e-02, 2.71191726e-03],
       [8.09958693e-01, 1.16482843e-02, 1.71156816e-02, ...,
        1.89405038e-03, 4.42854017e-02, 4.45955169e-03]])

In [ ]:
Y

array([[0.24716063],
       [0.01505958],
       [0.05828256],
       ...,
       [1.69761905],
       [0.02835498],
       [0.3540537 ]])

In [ ]:
#split the data for testing and training
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(964, 7) (241, 7) (964, 1) (241, 1)


## Building the Neural Network Model

In [ ]:
#Create a neutral network model
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten()),
model.add(tf.keras.layers.Dense(35, activation="relu")),
model.add(tf.keras.layers.Dense(30, activation="sigmoid")),
model.add(tf.keras.layers.Dense(25, activation="sigmoid")),
model.add(tf.keras.layers.Dense(1, activation="linear")),

#compile the model
model.compile(optimizer="adam",loss="mean_squared_error", metrics="accuracy")

In [ ]:
#train the model
model.fit(X_train, Y_train, epochs=450, batch_size=32)

Epoch 1/450
31/31 [==============================] - 1s 2ms/step - loss: 5.6654 - accuracy: 0.0000e+00
Epoch 2/450
31/31 [==============================] - 0s 2ms/step - loss: 5.3539 - accuracy: 0.0000e+00
Epoch 3/450
31/31 [==============================] - 0s 2ms/step - loss: 5.3597 - accuracy: 0.0000e+00
Epoch 4/450
31/31 [==============================] - 0s 2ms/step - loss: 5.3486 - accuracy: 0.0000e+00
Epoch 5/450
31/31 [==============================] - 0s 2ms/step - loss: 5.3588 - accuracy: 0.0000e+00
Epoch 6/450
31/31 [==============================] - 0s 2ms/step - loss: 5.3289 - accuracy: 0.0000e+00
Epoch 7/450
31/31 [==============================] - 0s 2ms/step - loss: 5.3269 - accuracy: 0.0000e+00
Epoch 8/450
31/31 [==============================] - 0s 2ms/step - loss: 5.3161 - accuracy: 0.0000e+00
Epoch 9/450
31/31 [==============================] - 0s 2ms/step - loss: 5.2968 - accuracy: 0.0000e+00
Epoch 10/450
31/31 [==============================] - 0s 2ms/step - loss:

## Predicting the test data

In [ ]:
Y_hat=model.predict(X_test)
Y_hat

array([[ 0.65796757],
       [ 0.5335035 ],
       [ 0.22054121],
       [ 9.707533  ],
       [ 1.8699102 ],
       [ 0.8091043 ],
       [ 0.35086983],
       [ 0.5050263 ],
       [ 0.66379684],
       [ 1.0004947 ],
       [ 3.3303366 ],
       [ 0.33273327],
       [ 0.37605405],
       [ 0.17995167],
       [ 0.1799511 ],
       [ 0.46443385],
       [ 0.17995554],
       [ 0.66055167],
       [ 0.1902653 ],
       [ 0.18991297],
       [ 0.5332361 ],
       [ 0.17995116],
       [ 1.8830633 ],
       [ 0.18570518],
       [ 0.17995104],
       [ 0.17995104],
       [ 0.48733228],
       [ 0.97644275],
       [ 0.33638883],
       [ 0.93072355],
       [ 0.18658045],
       [ 0.17995188],
       [ 0.22406578],
       [ 0.26811117],
       [ 0.33670792],
       [ 0.18199763],
       [ 0.46033794],
       [ 0.48199075],
       [ 0.66097957],
       [ 0.30713713],
       [ 0.17996532],
       [ 0.53630096],
       [ 0.27980438],
       [ 0.79235935],
       [ 0.91138554],
       [ 0

In [ ]:
Y_test

array([[2.29007634e-01],
       [2.96206888e-01],
       [1.75148633e-02],
       [2.07500000e+01],
       [2.20613948e+00],
       [3.88275862e-01],
       [1.32127261e-01],
       [5.07512120e-01],
       [1.43000000e-01],
       [6.04817416e-01],
       [8.17142857e+00],
       [7.45392492e-02],
       [2.45277406e-01],
       [2.86766623e-02],
       [2.64311594e-03],
       [1.00000000e-01],
       [4.77183833e-02],
       [1.86850091e-01],
       [4.74111727e-01],
       [1.56043011e-02],
       [5.59307829e-01],
       [2.95817491e-01],
       [1.33706897e+00],
       [9.94818653e-02],
       [8.04808476e-02],
       [3.94075282e-03],
       [9.07854137e-01],
       [1.18656388e+00],
       [2.30437750e-02],
       [3.70435468e-01],
       [3.46646571e-02],
       [7.54036667e-02],
       [6.55649038e-02],
       [1.81765236e-01],
       [3.82136330e-01],
       [3.42712843e-02],
       [2.33564951e-01],
       [2.82682103e-01],
       [5.21923621e-01],
       [2.92911541e-02],


# Predict the water stress values for 2022

In [ ]:
#Read and prepare the data
data_2022 = pd.read_csv('validation_data.csv')
data_2022

,Unnamed: 0,Year,Area,Agricultural water withdrawal as % of total water withdrawal,GDP per capita,Long-term average annual precipitation in volume,Municipal water withdrawal as % of total withdrawal,Population density,Total renewable water resources per capita,Total water withdrawal per capita
0,0,2022,Afghanistan,97.941104,656.180969,213.485220,0.993969,61.257508,969.975594,176.892287
1,1,2022,Albania,57.542793,5786.599868,41.381326,30.362635,101.094659,10689.844270,363.927480
2,2,2022,Algeria,58.977252,4659.461837,211.974923,38.004788,18.409220,233.342039,255.148373
3,3,2022,Angola,5.890727,4653.147601,1259.167000,53.707643,25.809087,3238.309847,21.052376
4,4,2022,Antigua and Barbuda,14.236588,16491.180460,0.453200,63.205424,231.527633,485.886409,131.255451
...,...,...,...,...,...,...,...,...,...,...
172,172,2022,Venezuela (Bolivarian Republic of),76.424646,9129.766093,1864.069566,21.187347,35.293496,32080.107670,819.597845
173,173,2022,Viet Nam,95.873180,2400.104517,602.640497,0.828757,304.226958,8166.541004,944.226983
174,174,2022,Yemen,90.544375,1217.420441,88.170990,7.490926,57.992638,56.084185,106.871156
175,175,2022,Zambia,72.242013,1861.535819,767.662200,18.899592,24.287317,4918.652720,67.947275


In [ ]:
#Extract the array for countries
area_2022=data_2022["Area"]
area_2022.to_frame()

,Area
0,Afghanistan
1,Albania
2,Algeria
3,Angola
4,Antigua and Barbuda
...,...
172,Venezuela (Bolivarian Republic of)
173,Viet Nam
174,Yemen
175,Zambia


##Plot for 2022

In [ ]:
X_2022_ori = data_2022[["Agricultural water withdrawal as % of total water withdrawal", "GDP per capita", "Long-term average annual precipitation in volume", "Municipal water withdrawal as % of total withdrawal", "Total renewable water resources per capita", "Total water withdrawal per capita", "Population density"]]
X_2022_scale = X_2022_ori.apply(lambda x: (x-x.min(axis=0)) / (x.max(axis=0)-x.min(axis=0)))
X_2022 = pd.DataFrame(X_2022_scale).to_numpy()
X_2022.shape

(177, 7)

In [ ]:
Y_2022=model.predict(X_2022)
#Y_2022=Y_2022.apply(lambda x: x*100)

In [ ]:
df1= pd.DataFrame(Y_2022, columns=["Water Stress"])
df2 = pd.concat([area_2022, df1], axis=1, join='inner')
df2.sort_values(by=["Water Stress"], ascending=False, inplace=True)

,Area,Water Stress
166,United Arab Emirates,20.466343
86,Kuwait,18.303040
139,Saudi Arabia,10.226875
79,Israel,7.828538
93,Libya,7.449485
...,...,...
123,Paraguay,0.126691
124,Peru,0.126691
88,Lao People's Democratic Republic,0.126691
170,Uruguay,0.126691


In [ ]:
country = df2['Area']
stress=df2['Water Stress']
fig = px.bar(df2,stress, y= country,orientation = 'h', title='Water Stress 2022')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()         